In [22]:
import os
os.chdir("/Users/jacksonwalters/Documents/GitHub/enefit-kaggle/predict-energy-behavior-of-prosumers/")

import IPython
import IPython.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from load_data import merged_df

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)

2.15.0


In [21]:
#load the dataset, slice a smaller amount for now
dataset = merged_df()

loading train data...
loading gas_prices...
loading electricity_prices...
loading forecast_weather...
merging train and gas_prices...
merging electricity_prices...
merging forecast_weather...


In [37]:
REDUCED_DATESET_SIZE = 1_000_000
df = dataset[:REDUCED_DATESET_SIZE].copy()

In [38]:
df

,county,is_business,product_type,target,is_consumption,prediction_datetime,data_block_id,row_id,prediction_unit_id,forecast_date,...,longitude,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,total_precipitation
2928,0,0,1,1.687,0,1.630555e+09,1,2928,0,1.630555e+09,...,21.7,13.691064,7.048120,0.000000,0.043335,0.835190,0.837845,0.971769,-10.245974,0.000090
2929,0,0,1,1.687,0,1.630555e+09,1,2928,0,1.630555e+09,...,22.2,11.859521,5.275171,0.000000,0.158173,0.370102,0.393387,0.535612,-5.105104,0.000000
2930,0,0,1,1.687,0,1.630555e+09,1,2928,0,1.630555e+09,...,22.7,13.141626,6.473901,0.000000,0.196793,0.158386,0.238663,1.305875,-7.770632,0.000018
2931,0,0,1,1.687,0,1.630555e+09,1,2928,0,1.630555e+09,...,23.2,13.239404,7.154810,0.000000,0.119019,0.164139,0.186142,1.239103,-9.215212,0.000080
2932,0,0,1,1.687,0,1.630555e+09,1,2928,0,1.630555e+09,...,23.7,13.527979,6.796289,0.000000,0.226669,0.194443,0.290100,1.273527,-9.503542,0.000048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202989,11,1,3,0.003,0,1.636607e+09,71,211922,48,1.636607e+09,...,23.2,7.990869,6.390527,0.621887,0.000000,0.000000,0.621887,5.392114,-0.097083,0.000000
1202990,11,1,3,0.003,0,1.636607e+09,71,211922,48,1.636607e+09,...,23.7,6.808130,5.782129,0.964600,0.053894,0.000000,0.966278,3.624780,0.146081,0.000000
1202991,11,1,3,0.003,0,1.636607e+09,71,211922,48,1.636607e+09,...,24.2,6.075220,5.552271,0.760956,0.347382,0.000000,0.847717,3.048364,0.618249,0.000000
1202992,11,1,3,0.003,0,1.636607e+09,71,211922,48,1.636607e+09,...,24.7,6.459497,6.082544,0.846924,0.609528,0.000000,0.942719,3.363793,0.810388,0.000000


In [39]:
#test/train split
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

#drop the target column 
train_labels = train_dataset['target']
test_labels = test_dataset['target']

train_features = train_dataset.drop(columns=['target'])
test_features = test_dataset.drop(columns=['target'])

num_features = len(test_features.columns.values)

In [40]:
#write feature_names to .csv file
import csv
feature_names = list(test_features.columns.values)
with open("../models/dnn_feature_names.csv", 'w') as f:
    write = csv.writer(f)
    write.writerow(feature_names)

In [41]:
#create a normalization layer
normalizer = tf.keras.layers.Normalization(input_shape=[num_features,])

#normalize the train_features by subtracting the mean and dividing by the std deviation
normalizer.adapt(np.array(train_features))

In [47]:
#build the three layer model with initial normalization layer
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(128, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001))
  return model

dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

#fit the model
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=1, epochs=2)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_5 (Normaliza  (None, 23)                47        
 tion)                                                           
                                                                 
 dense_21 (Dense)            (None, 128)               3072      
                                                                 
 dense_22 (Dense)            (None, 128)               16512     
                                                                 
 dense_23 (Dense)            (None, 1)                 129       
                                                                 
Total params: 19760 (77.19 KB)
Trainable params: 19713 (77.00 KB)
Non-trainable params: 47 (192.00 Byte)
_________________________________________________________________
Epoch 1/2
20000/20000 [==============================] - 89s 4ms/step - loss: 152

In [48]:
# save the model to disk via pickle
import pickle
dnn_model_filename = '../models/dnn_model.sav'
pickle.dump(dnn_model, open(dnn_model_filename, 'wb'))

In [49]:
#save model to disk via keras
dnn_model.save('../models/dnn_model.keras')

In [50]:
#save model to disk via legacy keras
dnn_model.save('../models/dnn_model.h5')

/Users/jacksonwalters/venv-metal/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
